In [1]:
import cv2
import sys
import time
import numpy as np
np.set_printoptions(suppress=True)
sys.path.append("../0.mc_utils/")

In [2]:
from common.tools_cv import draw_kp,im_rate_resize
from openpose.op_formats import NPMapOP25bToSMPL24
from pipeline.realtime_mview import server_cfg, MViewOP25bServer
from pipeline.mview_x2ds_to_A import DViewX2dsToA,CFG_MVIEW

from mqtt_player.unity3d_mqtt_player import Unity3DMqttPlayer,MQTTPLAYER_CFG

In [3]:
server_cfg.defrost()
server_cfg.host = '192.168.100.3'
server_cfg.freeze()
print(server_cfg)

host: 192.168.100.3
keepalive: 300
max_clients: 2
port: 12222


In [4]:
mview_to_A_cfg = CFG_MVIEW.clone()
mview_to_A_cfg.defrost()
mview_to_A_cfg.main_camera = '0'
mview_to_A_cfg.sub_camera = '1'
mview_to_A_cfg.cameras = ['0','1']
mview_to_A_cfg.K = [1100.,1100.,640.,360.]
mview_to_A_cfg.fusion.hmr.model="../12.models/mchmr2/20200510-hmrzeofusion-x2b5f2048s5-vposer0420-30.25.pth"
# mview_to_A_cfg.fusion.hmr.model="../12.models/mchmr2/20200528-hmrzeofusion-x2b5f2048s5-vposer0420.pth"
mview_to_A_cfg.hmr.model="../12.models/mchmr2/20200527-hmrzero-b4f2048s10-vposer0420-newmask.pth"
mview_to_A_cfg.freeze()
print(mview_to_A_cfg)

K: [1100.0, 1100.0, 640.0, 360.0]
MQTT_TX_UNITY:
  host: 127.0.0.1
  port: 12222
  topic: mc3d_msg
REID:
  cleanup_interval: 30
  max_cost: 200.0
  max_hp: 300
cameras: ['0', '1']
device: cuda:0
fusion:
  device: cuda:0
  fix_joints: [12, 22, 23]
  hmr:
    block_features: 2048
    blocks: 5
    final_features: 512
    in_features: 96
    model: ../12.models/mchmr2/20200510-hmrzeofusion-x2b5f2048s5-vposer0420-30.25.pth
    out_features: 43
  vposer:
    block_features: 1024
    blocks: 3
    latents: 32
    model: ../12.models/mchmr2/20200420-vposer1024-lt32b3.pth
hmr:
  block_features: 2048
  blocks: 4
  final_features: 512
  in_features: 48
  model: ../12.models/mchmr2/20200527-hmrzero-b4f2048s10-vposer0420-newmask.pth
  out_features: 48
main_camera: 0
sub_camera: 1
traj_offset: [0.0, -1.0, 0.0]


In [5]:
mview_x2ds_to_A = DViewX2dsToA(mview_to_A_cfg)
map_op25b_to_smpl24 = NPMapOP25bToSMPL24()
mqtt_player = Unity3DMqttPlayer(MQTTPLAYER_CFG)
server_m0 = MViewOP25bServer(server_cfg)

>>>Loaded HMR from:../12.models/mchmr2/20200527-hmrzero-b4f2048s10-vposer0420-newmask.pth
>>>Loaded HMR from:../12.models/mchmr2/20200510-hmrzeofusion-x2b5f2048s5-vposer0420-30.25.pth
>>>Loaded VPoser from:../12.models/mchmr2/20200420-vposer1024-lt32b3.pth


In [ ]:
frame_index=0
while True:
    x2ds_dict = server_m0.dict_multiple_kp2ds.copy()
#     server_m0.dict_multiple_kp2ds = {}
    smpl24_dict = {}
#     print(x2ds_dict)
    for k in x2ds_dict:
        ##step1.draw x2ds to image
        im = np.zeros((720,1280,3)).astype(np.uint8)
        kp2ds = x2ds_dict[k]
        if kp2ds is not None:
            smpl24_dict[k] = map_op25b_to_smpl24(kp2ds)[...,:2]
            draw_kp(im,smpl24_dict[k],fmt='smpl24',color=(0,255,0))
        ###step2.visualize
        cv2.imshow(str(k), im_rate_resize(im,0.5))
        cv2.waitKey(1)
#     print(smpl24_dict)
    ret = mview_x2ds_to_A.push(frame_index,smpl24_dict)
    if ret is not None:
        mqtt_player(ret["hids"],ret["A"].reshape(-1,384))
    frame_index += 1
# cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()

In [ ]:
# A = mqtt.Client()
# A.connect("192.168.1.26", 12222, 300)

# kp2ds = np.random.rand(1,25,3)
# kp2ds_time = float(datetime.datetime.now().strftime('%H%M%S%f')[0:9])
        
# M = kp2ds.shape[0]
# msg = struct.pack(f'1f1i{M*25*3}f', kp2ds_time, M, *kp2ds.reshape((-1)))

# A.publish('multiple_op_0', msg, 0)